# Data Clean for AU goods & service classification

## pre step finished at AWS Athena
export the data and save to CSV base on the following SQL
```
with t as (
    select application_number, goods, class_numbers
    from XXX
    where country_code='AU' and mark_status_normalized='LIVE REGISTRATION'
)
select application_number, cls, gs from t 
CROSS JOIN UNNEST(goods, class_numbers) as x(gs, cls)
```

the sample of output CSV:
```
"application_number","cls","gs"
"991907","16","Printed matter and publications;maps, guidebooks, street directories and business directories"
"991907","38","Telecommunications and online applications and services which access data using a telecommunications service, either permanently or intermittently;automatic land vehicle locating services"
```

NOTE:
for US, the earlier trademark's gs is not accurate, so select the good qulity data after 1990-01-01:
```
with t as (
    select application_number, goods, class_numbers
    from XXX
    where country_code='US' and mark_status_normalized='LIVE REGISTRATION' and registration_date > date('1990-01-01') 
)
select application_number, cls, gs from t 
CROSS JOIN UNNEST(goods, class_numbers) as x(gs, cls)
```

In [1]:
import pandas as pd
import pickle


In [2]:
%store -r ORI_CSV_INPUT_FILE
%store -r DF_INPUT_FILE


In [3]:
df_ori = pd.read_csv(ORI_CSV_INPUT_FILE)
df_ori.gs = df_ori.gs.str.split(";")
df_ori = df_ori.explode("gs")
df_dedup = df_ori[df_ori.duplicated(subset=['cls','gs']) == False]


## pre-process
### Normalization

In [25]:
# Normalization
df_dedup.gs = df_dedup.gs.str.lower()
# need to replace to SPACE instead of nothing
# i.e. like[this], cannot be likethis
df_dedup.gs = df_dedup.gs.str.replace(r"[^a-zA-Z0-9 ]", " ", regex=True)



/var/folders/vq/cs9cxfbn7lj71xw0q425_tnm0000gn/T/ipykernel_3993/271283872.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dedup.gs = df_dedup.gs.str.lower()
/var/folders/vq/cs9cxfbn7lj71xw0q425_tnm0000gn/T/ipykernel_3993/271283872.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dedup.gs = df_dedup.gs.str.replace(r"[^a-zA-Z0-9 ]", "", regex=True)


### stop word

In [ ]:
# Stop Word
from nltk.corpus import stopwords
regex_stop_words = [rf"\b{item}\b" for item in stopwords.words("english")]
regex_stop_words = "|".join(regex_stop_words)
df_dedup.gs = df_dedup.gs.str.replace(rf"{regex_stop_words}", "", regex=True)

### Stemming and Lemmatization(skip it due to the usage of word2vec)


### de-duplicate finally again begore import the data

In [27]:
df_dedup = df_dedup[df_dedup.duplicated(subset=['cls','gs']) == False]

### simple check

In [32]:
# check
# df_dedup.info()
# df_dedup.shape
pd.set_option('display.float_format', lambda x: '%.0f' % x)
df_dedup[["cls"]].describe()

,cls
count,940472
mean,22
std,14
min,0
25%,9
50%,21
75%,36
max,45


In [31]:
# NOTE for US
# should delete the 91 class as be the dirty data
df_dedup = df_dedup[df_dedup.cls!=91]

### output to pickle file

In [21]:
import os
import pickle
from pathlib import Path

pickle.dump(
    df_dedup,
    open(DF_INPUT_FILE,"wb")
)


# Temp code for testing

In [29]:
df_dedup.shape

(940474, 3)

In [15]:
df_dedup = df_ori[df_ori.duplicated(subset=['cls','gs']) == False]
# df_dedup.head()

In [24]:
import re
df_dedup[(df_dedup.cls==45) & (df_dedup.gs.str.contains("drink",flags=re.I))]

,application_number,cls,gs
259667,77167665,45,"Security services, namely, water vulnerability..."


In [33]:
df_dedup.groupby('cls').count()

,application_number,gs
cls,,
0,3273,3273
1,23612,23612
2,6146,6146
3,29316,29316
4,3616,3616
5,25366,25366
6,18921,18921
7,37056,37056
8,8808,8808
